# Q1: Install Spark and PySpark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder\
                    .master("local[*]")\
                    .appName("week5_homework")\
                    .getOrCreate()

spark.version

'3.5.0'

# Question 2:
FHV October 2019

In [3]:
df_fhv_tripdata_2019 = spark.read.csv("/home/Spark/fhv_tripdata_2019-10.csv", header = True, inferSchema = True)

In [4]:
df_fhv_repa = df_fhv_tripdata_2019.repartition(6)
df_fhv_repa.write.\
            format ("parquet").\
            mode("overwrite").save("/home/Spark/homework_data")

In [15]:
df_fhv_tripdata_2019.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



average size of the parquet files are 36MB

# Question 3:
Count records

In [6]:
df_fhv_tripdata_2019 = df_fhv_tripdata_2019.withColumn("pickup_date" , F.to_date("pickup_datetime"))
trips_15octoper_count = df_fhv_tripdata_2019.filter(df_fhv_tripdata_2019.pickup_date == "2019-10-15").count()
trips_15octoper_count

62610

# Question 4:
Longest trip for each day

In [21]:
df_fhv_tripdata_2019.withColumn("tripDuration", 
              df_fhv_tripdata_2019.dropOff_datetime.cast("long") -
              df_fhv_tripdata_2019.pickup_datetime.cast("long")/60).\
              groupby("pickup_date").\
              max("tripDuration").\
              orderBy("max(tripDuration)", ascending=False).limit(5).\
              show()
             

+-----------+-----------------+
|pickup_date|max(tripDuration)|
+-----------+-----------------+
| 2019-10-28|     3.81819798E9|
| 2019-10-11|     3.81678552E9|
| 2019-10-31|  1.86197695345E9|
| 2019-10-01|   1.7962610193E9|
| 2019-10-26|    1.577546074E9|
+-----------+-----------------+



# Question 5:
User Interfac
port: 8080
e

# Question 6:
Least frequent pickup location zone

In [23]:
df_zone = spark.read.csv("/home/Spark/taxi+_zone_lookup.csv", header = True, inferSchema=True)

In [26]:
df_zone.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [ ]:
df_fhv_tripdata_2019.join(df_zone, df_fhv_tripdata_2019.PUlocationID == df_zone.LocationID, "left").\
                     groupBy("zone").\
                     count().\
                     orderBy("count", ascending = False).\
                     limit(5).\
                     show()